# streamlit 밖 코랩에서 미리 실행

In [5]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q streamlit
!npm install localtunnel
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install sentence_transformers
!pip install streamlit-folium

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

up to date, audited 23 packages in 452ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-271cpeoc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-271cpeoc
  Resolved https://github.com/huggingface/transformers.git to commit 33eef992503689ba1af98090e26d3e98865b2a9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 19.3 MB/s eta 0:00:00


In [ ]:
!npm audit fix


up to date, audited 60 packages in 744ms

5 packages are looking for funding
  run `npm fund` for details

# npm audit report

axios  <=0.27.2
Severity: high
Axios vulnerable to Server-Side Request Forgery - https://github.com/advisories/GHSA-4w2v-q235-vp99
axios Inefficient Regular Expression Complexity vulnerability - https://github.com/advisories/GHSA-cph5-m8f7-6c5x
Axios Cross-Site Request Forgery Vulnerability - https://github.com/advisories/GHSA-wf5p-g6vw-rhxx
Depends on vulnerable versions of follow-redirects
fix available via `npm audit fix --force`
Will install localtunnel@2.0.2, which is a breaking change
node_modules/axios
  localtunnel  1.9.0 - 2.0.0
  Depends on vulnerable versions of axios
  Depends on vulnerable versions of debug
  Depends on vulnerable versions of yargs
  node_modules/localtunnel

debug  4.0.0 - 4.3.0
Regular Expression Denial of Service in debug - https://github.com/advisories/GHSA-gxpj-cx7g-858c
fix available via `npm audit fix --force`
Will install 

# config.py

In [68]:
%%writefile config.py

import streamlit as st
import pandas as pd
from PIL import Image, ImageOps
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import NearestNeighbors
import itertools
import requests
import urllib.parse
import requests
import folium
from folium.plugins import MarkerCluster
from folium import Map, Marker, PolyLine
import matplotlib.cm as cm
import matplotlib.colors as colors
import time
from shapely.geometry import MultiPoint
from streamlit_folium import folium_static

# Naver API 설정
NAVER_CLIENT_ID = "lufec25vd7"
NAVER_CLIENT_SECRET = "9tc3oII8TjHJgrdX9sgdgIEt8ieNeRuaacQG2ADB"

# 이미지 디렉토리 설정
image_directory = '/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/filtered_images_new'

# 필요한 데이터셋 모두 로드
final_similarity_df = pd.read_csv("/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/final_similarity_df_1112.csv")
with open('/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/aihub_embeddings_1112.json', 'r') as f:
    loaded_aihub_embeddings_list = json.load(f)
recommend_table = pd.read_csv('/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/recommend_table_1111.csv')
spot_table = pd.read_csv('/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/spot_table_1111.csv')
act_consume = pd.read_csv('/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/tn_activity_consume_his_활동소비내역_H.csv')
recommend_with_consume = pd.read_csv("/content/drive/MyDrive/contest/K-데이터사이언스 해커톤/recommend_with_consume_1111.csv")

# 모델 설정을 캐시하여 한번만 로드
@st.cache_resource
def load_model():
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )
    processor = AutoProcessor.from_pretrained('llava-hf/llava-1.5-7b-hf')
    model1 = LlavaForConditionalGeneration.from_pretrained(
        'llava-hf/llava-1.5-7b-hf',
        quantization_config=quantization_config,
        device_map="auto"
    )
    model2 = SentenceTransformer('bert-base-nli-mean-tokens')
    return processor, model1, model2


# 코사인 유사도 계산 함수
def calculate_pairwise_cosine_similarity(user_image, aihub_image_embeddings_list, category):
    _, _, model2 = load_model()
    sentences_user = user_image[category].tolist()
    sentences_user_embedding = model2.encode(sentences_user)  # user 이미지 텍스트 임베딩

    similarity_list = []
    # user 이미지의 각 임베딩과 aihub 임베딩을 비교
    for i in range(len(sentences_user_embedding)):
        for aihub_entry in aihub_image_embeddings_list:
            if aihub_entry['category'] == category:  # 카테고리 체크
                aihub_embedding = aihub_entry['embedding']  # 임베딩을 가져옴
                cosine_sim = cosine_similarity([sentences_user_embedding[i]], [aihub_embedding])[0][0]

                # aihub_entry에서 이미지 정보도 참조
                similarity_list.append({
                    'user_image': user_image['image'].iloc[i],
                    'aihub_image': aihub_entry['image'],
                    'response_column': category,
                    'cosine_similarity': cosine_sim
                })
    # 결과를 반환
    return similarity_list


# 선택된 이미지의 유사도 정보를 생성하는 함수
def generate_similarity_info(selected_images, final_similarity_df, similar_images):
    similarity_info = []

    # 선택된 이미지의 각 response에 대한 유사도 정보 저장
    for response, selected_image in selected_images.items():
        # 선택된 이미지와 response가 일치하는 경우에 대해서만 유사도 정보를 가져오기
        selected_similarities = final_similarity_df.loc[
            ((final_similarity_df['user_image'] == selected_image) | (final_similarity_df['aihub_image'] == selected_image)) &
            (final_similarity_df['response_column'] == response)
        ]

        for _, row in selected_similarities.iterrows():
            other_image = row['aihub_image'] if row['user_image'] == selected_image else row['user_image']
            similarity_info.append({
                'response': response,
                'selected_image': selected_image,
                'other_image': other_image,
                'cosine_similarity': row['cosine_similarity']
            })

    # 자기 자신과의 유사도 추가
    for response, df in similar_images.items():
        for selected_image in selected_images.values():
            # 해당 response에 선택된 이미지가 포함되는 경우에만 자기 자신과의 유사도 추가
            if selected_image in df['aihub_image'].values:
                similarity_info.append({
                    'response': response,
                    'selected_image': selected_image,
                    'other_image': selected_image,
                    'cosine_similarity': 1.0  # 자기 자신과의 유사도는 1로 가정
                })

    # 각 other_image에 대해 cosine similarity 값을 모아 평균 계산
    average_similarities = {}
    # similarity_info에서 other_image별로 cosine similarity 모음
    for info in similarity_info:
        other_image = info['other_image']
        if other_image not in average_similarities:
            average_similarities[other_image] = []
        average_similarities[other_image].append(info['cosine_similarity'])

    # 각 other_image의 평균 유사도 계산
    for image, similarities in average_similarities.items():
        average_similarities[image] = sum(similarities) / len(similarities)

    return average_similarities


# 가장 유사한 이미지 경로를 반환하는 함수
def get_most_similar_image_path(average_similarities):
    if average_similarities:
        most_similar_image = max(average_similarities, key=average_similarities.get)
        most_similar_image_path = os.path.join(image_directory, most_similar_image)
        return most_similar_image_path
    else:
        return None

# 유사도 계산 함수
def similarity(new_traveler, recommend_table_near):
    # 명목형 변수 유사도 계산: 같으면 각각 0.5, 다르면 0
    nominal_similarity = (recommend_table_near[['GENDER', 'TRAVEL_MOTIVE_1']] == new_traveler[[0,3]]).astype(int)

    age_grp = pd.to_numeric(recommend_table_near['AGE_GRP'], errors='coerce').fillna(0).astype(int)
    new_traveler_age_grp = int(new_traveler[1])  # new_traveler의 나이 그룹도 정수로 변환
    age_diff = np.abs(age_grp - new_traveler_age_grp)

    travel_style = pd.to_numeric(recommend_table_near['TRAVEL_STYL_1'], errors='coerce').fillna(0).astype(int)
    new_traveler_travel_style = int(new_traveler[2])
    # 여행 스타일 차이 계산
    travel_style_diff = np.abs(travel_style - new_traveler_travel_style)

    ordinal_similarity = pd.DataFrame({
        'AGE_SIM': 1 - (age_diff / age_diff.max()),  # 정규화
        'STYLE_SIM': 1 - (travel_style_diff / travel_style_diff.max())  # 정규화
    })

    # 전체 유사도: 명목형과 순서형 유사도의 평균
    similarity = (nominal_similarity.mean(axis=1) + ordinal_similarity.mean(axis=1)) / 2
    return similarity


# 추천 함수
def recommend(spot, new_traveler, metric, recommend_with_consume=None, n=5):
    spot_coord = spot[['X_COORD', 'Y_COORD']].values
    coords = spot_table[['X_COORD', 'Y_COORD']]

    if metric == 'knn':
        K = 51
        knn = NearestNeighbors(n_neighbors=K)
        knn.fit(coords)

        spot_coord = spot_coord.reshape(1, -1)
        dist, idx = knn.kneighbors(spot_coord)
        idx = idx.reshape(-1)

        near_place = spot_table.loc[idx]
        near_place['Distance'] = dist.reshape(-1)
    else:
        R = 0.1
        spot_coord = spot_coord.reshape(-1)
        x, y = spot_coord[0], spot_coord[1]

        near_place = spot_table.copy()
        near_place['Distance'] = np.sqrt((near_place['X_COORD'] - x)**2 + (near_place['Y_COORD'] - y)**2)
        near_place = near_place[near_place.Distance < R]

    near_POI = near_place['POI_ID']
    recommend_table_near = spot_table[spot_table.POI_ID.isin(near_POI)].reset_index(drop=True)
    similar_travelers = recommend_table_near.copy()
    similar_travelers['SIMILARITY'] = similarity(new_traveler, recommend_table_near)

    max_idx = similar_travelers.groupby('POI_ID')['SIMILARITY'].idxmax()

    result = similar_travelers.loc[max_idx]
    result = result[result.ratings >= 4]

    result = result.sort_values(by='SIMILARITY', ascending=False).iloc[:n]

    return result

# 특정 장소와 소비 내역 데이터를 기반으로 장소별 평균 결제 시간을 계산하여 추가하는 함수
def locations_with_time(locations, act_consume):

    # 새로운 컬럼 생성
    locations['avg_payment_time'] = None

    for idx, row in locations.iterrows():
        road_nm_addr = row['ROAD_NM_ADDR']

        # act_consume 데이터프레임에서 해당 ROAD_NM_ADDR를 가지는 행 필터링
        matching_rows = act_consume[act_consume['ROAD_NM_ADDR'] == road_nm_addr]

        if not matching_rows.empty:
            # PAYMENT_DT에서 시간 부분 (HH:MM)만 추출하여 datetime 형식으로 변환
            matching_rows['PAYMENT_TIME'] = pd.to_datetime(matching_rows['PAYMENT_DT'], errors='coerce').dt.strftime('%H:%M')

            # 유효한 시간 값만 선택 (NaN 값 제외)
            valid_times = matching_rows['PAYMENT_TIME'].dropna()
            if not valid_times.empty:
                # 시간 부분을 'HH:MM' 포맷의 문자열로 유지하여 hour와 minute을 분리 후 초로 변환
                valid_times_minutes = valid_times.apply(lambda t: int(t[:2]) * 60 + int(t[3:]))
                avg_minutes = valid_times_minutes.mean()

                # 평균 분을 'HH:MM' 형식으로 변환
                avg_hours = int(avg_minutes // 60)
                avg_minutes = int(avg_minutes % 60)

                # 평균 시간을 'HH:MM' 형식으로 저장
                avg_payment_time = f"{avg_hours:02}:{avg_minutes:02}"
                locations.at[idx, 'avg_payment_time'] = avg_payment_time
            else:
                # 유효한 시간이 없을 경우 None 할당
                locations.at[idx, 'avg_payment_time'] = None

    return locations


def extract_dining(locations, act_consume):
    # 각 장소별 결과를 저장할 리스트
    location_results = []

    # locations의 각 행에 대해 반복
    for _, location_row in locations.iterrows():
        target_travel_id = location_row['TRAVEL_ID']
        target_road_nm_addr = location_row['ROAD_NM_ADDR']
        poi_name = location_row['POI_NM']

        # act_consume에서 해당 TRAVEL_ID를 가지는 모든 데이터 가져오기
        filtered_df = act_consume[act_consume['TRAVEL_ID'] == target_travel_id]

        # PAYMENT_DT 순으로 정렬
        sorted_df = filtered_df.sort_values(by='PAYMENT_DT').reset_index(drop=True)

        # 기준이 되는 행 찾기 (ROAD_NM_ADDR가 동일한 첫 번째 행)
        reference_row = sorted_df[sorted_df['ROAD_NM_ADDR'] == target_road_nm_addr].head(1)

        if reference_row.empty:
            # 기준 행이 없는 경우 빈 데이터프레임을 결과 리스트에 추가하고 다음으로 넘어감
            location_results.append((poi_name, pd.DataFrame()))
            continue

        # 기준 행의 인덱스 가져오기
        ref_index = reference_row.index[0]

        # 전후로 ACTIVITY_TYPE_CD 값이 1인 데이터를 찾기
        activity_data = []

        # 이전 행에서 최대 2개의 행 찾기
        count = 0
        for i in range(ref_index - 1, -1, -1):
            if sorted_df.loc[i, 'ACTIVITY_TYPE_CD'] == 1:
                activity_data.append(sorted_df.loc[i])
                count += 1
                if count == 2:
                    break

        # 이후 행에서 최대 2개의 행 찾기
        count = 0
        for i in range(ref_index + 1, len(sorted_df)):
            if sorted_df.loc[i, 'ACTIVITY_TYPE_CD'] == 1:
                activity_data.append(sorted_df.loc[i])
                count += 1
                if count == 2:
                    break

        # 추출한 데이터를 하나의 데이터프레임으로 변환하여 리스트에 (POI_NM, 데이터프레임) 형태로 저장
        result_df = pd.DataFrame(activity_data)
        location_results.append((poi_name, result_df))

    return location_results

def path_optimizer(spot, locations):

    headers = {
    "X-NCP-APIGW-API-KEY-ID": NAVER_CLIENT_ID,
    "X-NCP-APIGW-API-KEY": NAVER_CLIENT_SECRET
    }

    # 중복된 좌표 제거 및 초기화
    locations = locations.drop_duplicates(subset=['X_COORD', 'Y_COORD'], keep='first')
    locations = locations[~locations['VISIT_AREA_NM'].isin(spot['VISIT_AREA_NM'])].reset_index(drop=True)

    spot_coord = spot[['X_COORD', 'Y_COORD']]
    coordinates = pd.concat([spot_coord, locations[['X_COORD', 'Y_COORD']]], axis=0, ignore_index=True).values
    num_locations = len(coordinates)
    distance_matrix = np.zeros((num_locations, num_locations))
    duration_matrix = np.zeros((num_locations, num_locations))

    # 거리 및 소요 시간 계산
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                start = f"{coordinates[i][0]},{coordinates[i][1]}"
                end = f"{coordinates[j][0]},{coordinates[j][1]}"
                url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start}&goal={end}&option=trafast"
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    result = response.json()
                    if result['route']:
                        distance = result['route']['trafast'][0]['summary']['distance']
                        duration = result['route']['trafast'][0]['summary']['duration']
                        distance_matrix[i][j] = distance
                        duration_matrix[i][j] = duration
                    else:
                        st.write(f"경로 없음: {i} -> {j}")
                else:
                    st.write(f"API 요청 실패: {response.status_code}")

    # 최적 경로 찾기
    min_path, min_duration, total_distance = None, float('inf'), 0
    for perm in itertools.permutations(range(1, num_locations)):
        current_path = [0] + list(perm)
        current_duration, current_distance = 0, 0
        for k in range(len(current_path) - 1):
            current_duration += duration_matrix[current_path[k]][current_path[k + 1]]
            current_distance += distance_matrix[current_path[k]][current_path[k + 1]]

        if current_duration < min_duration:
            min_duration = current_duration
            total_distance = current_distance
            min_path = current_path

    spot_df = pd.DataFrame(spot)
    ind = min_path[1:]
    ind = [x - 1 for x in ind]
    result = pd.concat([spot_df, locations.loc[ind]]).reset_index(drop=True)
    st.subheader("✨총 이동 소요 시간")
    st.write(f"{int(min_duration / 60000)}분")
    st.subheader("✨총 거리")
    st.write(f"{total_distance}m")
    #st.write("여정 순서 및 정보")
    #result2 = result[['POI_NM', 'ratings', 'SIMILARITY']]
    #st.dataframe(result2)

    # 각 POI 간의 소요 시간을 포함한 데이터프레임 생성 (도착지 기준)
    travel_times = []
    for i in range(len(min_path) - 1):
        start_index = min_path[i]
        end_index = min_path[i + 1]
        travel_time = round(duration_matrix[start_index, end_index] / 60000)  # ms -> min

        # 출발지와 도착지를 올바르게 설정
        start_location = locations.iloc[start_index - 1]['VISIT_AREA_NM'] if start_index > 0 else spot.iloc[0]['VISIT_AREA_NM']
        end_location = locations.iloc[end_index - 1]['VISIT_AREA_NM'] if end_index > 0 else spot.iloc[0]['VISIT_AREA_NM']
        travel_times.append({
            '출발': start_location,
            '도착': end_location,
            '소요시간': travel_time
        })

    # 새로운 데이터프레임 생성
    st.subheader("✨여정 순서 및 정보")
    travel_df = pd.DataFrame(travel_times)
    st.table(travel_df)

    return result, min_duration, total_distance, duration_matrix, distance_matrix


# 각 POI_NM별로 데이터를 처리하여 하나의 DataFrame으로 변환
def convert_dining_to_multiindex(dining):
    data = []
    for poi_name, dining_df in dining:
        dining_df['POI_NM'] = poi_name  # 각 DataFrame에 POI_NM 컬럼 추가
        data.append(dining_df)

    # 모든 데이터를 하나로 합침
    combined_df = pd.concat(data, ignore_index=True)

    # MultiIndex 생성
    combined_df.set_index(['POI_NM', combined_df.groupby('POI_NM').cumcount() + 1], inplace=True)

    return combined_df


# 경로를 카카오 API로 받아오는 함수
def get_route(origin, destination):
    # 카카오 API 키 설정
    kakao_headers = {
        "Authorization": "KakaoAK fcf14da2e76244a746666e84a90b0900"
    }
    url = f"https://apis-navi.kakaomobility.com/v1/directions?origin={origin}&destination={destination}&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"
    try:
        response = requests.get(url, headers=kakao_headers)
        response.raise_for_status()
        route_data = response.json()
        return route_data
    except requests.exceptions.RequestException as e:
        print(f"경로 API 요청 실패: {e}")
        return None

# 네이버 API로 이미지 검색 함수
def search_image(visit_area_nm, max_retries=10):
    # visit_area_nm을 URL 인코딩
    encoded_query = urllib.parse.quote(visit_area_nm)  # 한글을 URL 인코딩
    url = f"https://openapi.naver.com/v1/search/image.json?query={encoded_query}&display=1&start=1&sort=sim"
    for attempt in range(max_retries):
        try:
            headers = {
                "X-Naver-Client-Id": "pbn5UWRRNtptJlgjHgyy",
                "X-Naver-Client-Secret": "onAoSavYbe"
            }
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # 상태 코드 200이 아닐 경우 예외 발생
            result = response.json()
            if 'items' in result and result['items']:
                image_url = result['items'][0]['link']
                return image_url
            else:
                print(f"No images found for query: {visit_area_nm}")
                return None
        except requests.exceptions.RequestException as e:
            st.write(f"API 요청 실패: {e}")
            if attempt < max_retries - 1:
                st.write(f"재시도 중... {attempt + 1}/{max_retries}")
                time.sleep(2)  # 잠시 기다렸다가 재시도
            else:
                st.write(f"최대 재시도 횟수({max_retries})를 초과했습니다.")
                return None
        except ValueError as e:
            st.write(f"JSON 파싱 실패: {e}")
            return None
    return None

# 지도에 경로를 시각화하는 함수
def visualize_path_on_map(route_data, route_map, color):
    if route_data and 'routes' in route_data and len(route_data['routes']) > 0:
        route_coords = route_data['routes'][0]['sections'][0]['roads']
        all_vertexes = []
        for road in route_coords:
            vertexes = road['vertexes']
            for i in range(0, len(vertexes), 2):
                all_vertexes.append([vertexes[i + 1], vertexes[i]])
        folium.PolyLine(
            locations=all_vertexes,
            color=color,
            weight=5,
            opacity=0.7,
            popup="경로"
        ).add_to(route_map)

def create_map_with_markers_and_ordered_paths(path, poi_store_mapping):
    route_map = folium.Map(location=(path.iloc[0]['Y_COORD'], path.iloc[0]['X_COORD']), zoom_start=12)

    # 전체 경로 색상은 blue로 통일
    colormap = cm.get_cmap('rainbow', len(path) - 1)
    norm = colors.Normalize(vmin=0, vmax=len(path) - 2)

    for i in range(len(path) - 1):
        origin = f"{path.iloc[i]['X_COORD']},{path.iloc[i]['Y_COORD']}"
        destination = f"{path.iloc[i + 1]['X_COORD']},{path.iloc[i + 1]['Y_COORD']}"
        # 카카오 API 경로 데이터 가져오기
        route_data = get_route(origin, destination)
        route_color = 'blue'  # 전체 경로는 blue로 통일

        if route_data:
            visualize_path_on_map(route_data, route_map, color=route_color)

        # 이미지 검색 및 마커 표시
        poi_name = path.iloc[i]['VISIT_AREA_NM']
        image_url = search_image(poi_name)

        # 'STORE_NM' 목록 추출
        store_names = poi_store_mapping.loc[poi_store_mapping['POI_NM'] == poi_name, 'STORE_NM']
        store_names_list = store_names.iloc[0] if not store_names.empty else []

        # STORE_NM 정보를 HTML로 변환
        stores_html = "<br>".join(store_names_list) if store_names_list else "No stores available"

        # 팝업 HTML 구성
        popup_html = f"<b>{i+1}. {poi_name}</b><br>"
        if image_url:
            popup_html += f"<img src='{image_url}' width='100'><br>"
        popup_html += f"식사 장소로 이곳은 어떤가요?:<br>{stores_html}"

        # 마커 추가
        folium.Marker(
            location=(path.iloc[i]['Y_COORD'], path.iloc[i]['X_COORD']),
            popup=folium.Popup(popup_html, max_width=200),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(route_map)

    # 마지막 장소에 대한 마커 표시
    last_poi_name = path.iloc[-1]['VISIT_AREA_NM']
    last_image_url = search_image(last_poi_name)
    last_popup_html = f"<b>{len(path)}. {last_poi_name}</b><br>"
    if last_image_url:
        last_popup_html += f"<img src='{last_image_url}' width='100'><br>"

    folium.Marker(
        location=(path.iloc[-1]['Y_COORD'], path.iloc[-1]['X_COORD']),
        popup=folium.Popup(last_popup_html, max_width=200),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(route_map)

    # 특정 장소 클릭 시 경로 표시
    def on_marker_click(marker, place):
        # 출발지와 도착지를 구분하여 경로 색상 변경
        start_point = marker.location
        for i in range(len(path) - 1):
            origin = f"{path.iloc[i]['X_COORD']},{path.iloc[i]['Y_COORD']}"
            destination = f"{path.iloc[i + 1]['X_COORD']},{path.iloc[i + 1]['Y_COORD']}"
            route_data = get_route(origin, destination)

            # 출발지와 도착지일 경우 색상 다르게
            route_color = 'red' if place == 'start' else 'green'

            if route_data:
                visualize_path_on_map(route_data, route_map, color=route_color)

    folium_static(route_map)

    return route_map

Overwriting config.py


# app.py

In [20]:
''' 원본
%%writefile app.py

import os
import streamlit as st
from datetime import datetime
from PIL import Image, ImageOps
import pandas as pd
import numpy as np
# config.py에서 가져오기
from config import (
    load_model,
    calculate_pairwise_cosine_similarity,
    generate_similarity_info,
    get_most_similar_image_path,
    loaded_aihub_embeddings_list,
    image_directory,
    recommend_table,
    locations_with_time,
    extract_dining,
    recommend,
    path_optimizer,
    spot_table,
    act_consume,
    recommend_with_consume,
    convert_dining_to_multiindex,
    create_map_with_markers_and_ordered_paths
)
from streamlit_folium import st_folium

# 모델 로드 (캐시 사용)
processor, model1, model2 = load_model()


# 유저 업로드 이미지 저장 함수
def save_uploaded_file(directory, file) :
    if not os.path.exists(directory) :
        os.makedirs(directory)
    with open(os.path.join(directory, file.name), 'wb') as f:
        f.write(file.getbuffer())
    return os.path.join(directory, file.name)


# 프롬프트에 대한 응답 생성 함수
def generate_responses(image_path):
    prompts = [
        "USER: <image>\n What other travel destinations or travel activities do you have that go well with this place? Please list it in words. \nASSISTANT:",
        "USER: <image>\n Please express the season that goes best with this picture and why. \nASSISTANT:",
        "USER: <image>\n Tell me about the mood, touch and feelings of this image in detail. Use as many adjectives as possible to express them. Reduce objective descriptions and add a lot of subjective expressions. \nASSISTANT:",
        "USER: <image>\n Describe this image objectively and realistically using the objects in this image. Do not include subjective words such as the mood, touch, atmosphere and feelings. \nASSISTANT:"
        ]
    responses = []
    image = Image.open(image_path)
    for prompt in prompts:
        inputs = processor([image], [prompt], padding=True, return_tensors="pt").to("cuda")
        output = model1.generate(**inputs, max_new_tokens=100)
        generated_text = processor.batch_decode(output, skip_special_tokens=True)
        response = generated_text[0].split("ASSISTANT:")[-1].strip() if "ASSISTANT:" in generated_text[0] else generated_text[0].strip()
        responses.append(response)
    return responses

# 메인 함수
def main():
    st.title('사진 속 추억과 다시 만나는 여행✨')  # 웹사이트 메인 제목
    file = st.file_uploader('여행 사진을 업로드해 주세요.', type=['jpg','jpeg','png'])

    if file is not None :
        current_time = datetime.now().isoformat().replace(':', '_')
        file.name = current_time + '.jpg'
        image_path = save_uploaded_file('tmp', file)  # 코랩 좌측 열면 tmp라는 폴더가 생기고 거기에 사진 저장됨

        # 임시 공간 생성 및 메시지 표시
        message_placeholder = st.empty()
        message_placeholder.write("🕵️사진을 분석하는 중이에요!")

        image = Image.open(file)
        image = ImageOps.exif_transpose(image)  # EXIF 정보를 이용해 올바른 방향으로 회전
        st.image(image.resize((200, 250)))

        responses = generate_responses(image_path)
        categories = ['response_1', 'response_2', 'response_3', 'response_4']
        category_names = {
                'response_1': '비슷한 여행지',
                'response_2': '계절',
                'response_3': '감정',
                'response_4': '객관적 묘사'
            }
        df_responses = pd.DataFrame([responses], columns=categories)
        df_responses['image'] = file.name

        message_placeholder.write("🔎비슷한 사진을 찾고 있어요...")

        all_similarity_results = [
                result
                for category in categories
                for result in calculate_pairwise_cosine_similarity(df_responses, loaded_aihub_embeddings_list, category)
            ]
        final_similarity_df_test = pd.DataFrame(all_similarity_results)
        similar_images = {
                category: final_similarity_df_test[final_similarity_df_test['response_column'] == category]
                .nlargest(3, 'cosine_similarity')
                .reset_index(drop=True)
                for category in categories
            }

        # 세션 상태 초기화
        for key, default in {
            "selected_images": {category: None for category in categories},
            "form_submitted": False,
            "profile_submitted": False,
            "new_traveler": None,
            "num_recommendations": 5,
            "user_profile": {}
        }.items():
            if key not in st.session_state:
                st.session_state[key] = default

        message_placeholder.write("💖마음에 드는 사진을 골라주세요.")

        # 이미지 선택 폼
        if not st.session_state['form_submitted']:
            with st.form("selection_form"):
                for category in categories:
                    display_name = category_names.get(category, category)
                    st.write(f"✅{display_name}")
                    cols = st.columns(3)
                    options = []
                    for i, (col, (_, row)) in enumerate(zip(cols, similar_images[category].iterrows())):
                        filename = row['aihub_image']
                        image_path = os.path.join(image_directory, filename)
                        with col:
                            if os.path.exists(image_path):
                                img = Image.open(image_path)
                                img = ImageOps.exif_transpose(img)
                                img = img.resize((200, 200))
                                st.image(img, caption=f"{i + 1}", use_container_width=True)
                            else:
                                st.write(f"{i + 1}. 이미지 파일을 찾을 수 없습니다: {filename}")
                            options.append((f"{i + 1}", filename))
                    # 라디오 버튼을 통해 이미지 선택, 각 라디오의 키는 카테고리와 인덱스를 조합해 유일하게 설정
                    selected_option = st.radio(
                        "💖마음에 드는 사진을 골라주세요.",
                        [opt[0] for opt in options],
                        key = f"{category}_radio_{i}",
                        horizontal = True
                    )
                    # 선택된 파일명을 session_state에 저장
                    try:
                        st.session_state.selected_images[category] = next(
                            filename for label, filename in options if label == selected_option
                        )
                    except StopIteration:
                        st.write("선택된 이미지가 없습니다.")
                if st.form_submit_button("선택 완료"):
                    st.session_state.form_submitted = True

        message_placeholder.write("🎉가장 비슷한 추억을 찾았어요!")
        # 폼 제출 후 최종 유사 이미지 계산
        if st.session_state["form_submitted"] and not st.session_state["profile_submitted"]:
            # 유사도 정보 생성 및 평균 유사도 계산
            average_similarities = generate_similarity_info(
                    st.session_state.selected_images,
                    final_similarity_df_test,
                    similar_images
                )
            most_similar_image_path = get_most_similar_image_path(average_similarities)
            del st.session_state['form_submitted']
            if most_similar_image_path and os.path.exists(most_similar_image_path):
                most_similar_image = os.path.basename(most_similar_image_path)
                image = Image.open(most_similar_image_path)
                image = ImageOps.exif_transpose(image)
                image = image.resize((200, 250))
                st.image(image)
                spot = recommend_table.loc[recommend_table['PHOTO_FILE_NM'] == most_similar_image]
                if spot.empty:
                    st.write('추천 장소를 찾을 수 없어요.')
                else:
                    st.table(spot)

                    # 사용자 특성 입력 폼
                    travel_style_mapping = {
                        '자연 매우 선호': 1,
                        '자연 중간 선호': 2,
                        '자연 약간 선호': 3,
                        '중립': 4,
                        '도시 약간 선호': 5,
                        '도시 중간 선호': 6,
                        '도시 매우 선호': 7
                    }
                    travel_motive_mapping = {
                        '일상적인 환경 및 역할에서의 탈출, 지루함 탈피': 1,
                        '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식': 2,
                        '여행 동반자와의 친밀감 및 유대감 증진': 3,
                        '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기': 4,
                        'SNS 사진 등록 등 과시': 5,
                        '운동, 건강 증진 및 충전': 6,
                        '새로운 경험 추구': 7,
                        '역사 탐방, 문화적 경험 등 교육적 동기': 8,
                        '특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)': 9,
                        '기타': 10
                    }
                    st.subheader("본인에 대해 알려주세요")
                    with st.form("user_profile_form"):
                        gender = st.radio("성별", [0, 1], format_func=lambda x: "남성" if x == 0 else "여성",
                                            horizontal=True)
                        age_group_label = st.radio("나이",
                        ['20대', '30대', '40대', '50대', '60대'], horizontal=True)
                        age_group = int(age_group_label[:-1])
                        travel_style_label = st.radio("자연 vs 도시", list(travel_style_mapping.keys()))
                        travel_style = travel_style_mapping[travel_style_label]
                        travel_motive_label = st.radio("여행 동기", list(travel_motive_mapping.keys()))
                        travel_motive = travel_motive_mapping[travel_motive_label]
                        num_recommendations = st.slider("추천받고 싶은 장소 개수", min_value=3, max_value=10, value=5, step=1)

                        if st.form_submit_button("선택 완료"):
                            st.session_state.user_profile = {
                                    "GENDER": gender,
                                    "AGE_GRP": age_group,
                                    "TRAVEL_STYL_1": travel_style,
                                    "TRAVEL_MOTIVE_1": travel_motive,
                                    "NUM_RECOMMENDATIONS": num_recommendations
                                }
                            st.session_state.profile_submitted = True
                            st.write("사용자 프로필이 저장되었습니다.")
                            st.session_state.new_traveler = np.array([gender, age_group, travel_style, travel_motive])
                            st.session_state.num_recommendations = num_recommendations
        st.session_state.profile_submitted = True
        # 추천 장소 및 추가 정보 추출
        if st.session_state.profile_submitted:
            locations = recommend(
                            spot,
                            st.session_state.new_traveler,
                            metric = 'knn',
                            n = st.session_state.num_recommendations,
                            recommend_with_consume = recommend_with_consume
                        )
            locations_time = locations_with_time(locations, act_consume)
            locations_time_print = locations_time[['VISIT_AREA_NM',
                                                   'ROAD_NM_ADDR',
                                                   'ratings']]
            locations_time_print.columns = ['이름', '주소', '별점']

            st.subheader("추천 장소")
            st.table(locations_time_print)

            dining = extract_dining(locations_time, act_consume)


            # 경로 최적화 수행
            path, duration, dist, duration_matrix, _ = path_optimizer(spot, locations)

            # dining을 MultiIndex DataFrame으로 변환
            dining_multiindex_df = convert_dining_to_multiindex(dining)
            poi_store_mapping = dining_multiindex_df.groupby('POI_NM')['STORE_NM'].apply(list).reset_index()
            st.table(poi_store_mapping)

            # 경로와 마커가 포함된 지도 생성
            route_map = create_map_with_markers_and_ordered_paths(path, poi_store_mapping)

if __name__=='__main__' :
    main()

Overwriting app.py


## 원본

In [ ]:
''' 원본
# 파일을 업로드 하는 방법
%%writefile app.py

import os
import streamlit as st
from datetime import datetime
from PIL import Image, ImageOps
import pandas as pd
import numpy as np
# config.py에서 가져오기
from config import (
    load_model,
    calculate_pairwise_cosine_similarity,
    generate_similarity_info,
    get_most_similar_image_path,
    loaded_aihub_embeddings_list,
    image_directory,
    recommend_table,
    locations_with_time,
    extract_dining,
    recommend,
    path_optimizer,
    spot_table,
    act_consume,
    recommend_with_consume,
    convert_dining_to_multiindex,
    create_map_with_markers_and_ordered_paths
)
from streamlit_folium import st_folium

# 모델 로드 (캐시 사용)
processor, model1, model2 = load_model()


# 유저 업로드 이미지 저장 함수
def save_uploaded_file(directory, file) :
    if not os.path.exists(directory) :
        os.makedirs(directory)
    with open(os.path.join(directory, file.name), 'wb') as f:
        f.write(file.getbuffer())
    return os.path.join(directory, file.name)


# 프롬프트에 대한 응답 생성 함수
def generate_responses(image_path):
    prompts = [
        "USER: <image>\n What other travel destinations or travel activities do you have that go well with this place? Please list it in words. \nASSISTANT:",
        "USER: <image>\n Please express the season that goes best with this picture and why. \nASSISTANT:",
        "USER: <image>\n Tell me about the mood, touch and feelings of this image in detail. Use as many adjectives as possible to express them. Reduce objective descriptions and add a lot of subjective expressions. \nASSISTANT:",
        "USER: <image>\n Describe this image objectively and realistically using the objects in this image. Do not include subjective words such as the mood, touch, atmosphere and feelings. \nASSISTANT:"
        ]
    responses = []
    image = Image.open(image_path)

    for prompt in prompts:
        inputs = processor([image], [prompt], padding=True, return_tensors="pt").to("cuda")
        output = model1.generate(**inputs, max_new_tokens=100)
        generated_text = processor.batch_decode(output, skip_special_tokens=True)
        response = generated_text[0].split("ASSISTANT:")[-1].strip() if "ASSISTANT:" in generated_text[0] else generated_text[0].strip()
        responses.append(response)

    return responses

# 메인 함수
def main():
    st.title('사진 속 추억과 다시 만나는 여행')  # 웹사이트 메인 제목
    file = st.file_uploader('여행 사진을 업로드해 주세요.', type=['jpg','jpeg','png'])

    if file is not None :
        current_time = datetime.now().isoformat().replace(':', '_')
        file.name = current_time + '.jpg'
        image_path = save_uploaded_file('tmp', file)  # 코랩 좌측 열면 tmp라는 폴더가 생기고 거기에 사진 저장됨

        # 임시 공간 생성 및 메시지 표시
        message_placeholder = st.empty()

        image = Image.open(file)
        image = ImageOps.exif_transpose(image)  # EXIF 정보를 이용해 올바른 방향으로 회전
        st.image(image.resize((250, 300)))


        message_placeholder.write("올려주신 사진을 분석하는 중이에요!")

        responses = generate_responses(image_path)
        categories = ['response_1', 'response_2', 'response_3', 'response_4']
        category_names = {
                'response_1': '비슷한 여행지',
                'response_2': '계절',
                'response_3': '감정',
                'response_4': '객관적 묘사'
            }
        df_responses = pd.DataFrame([responses], columns=categories)
        df_responses['image'] = file.name

        message_placeholder.write("비슷한 사진을 찾고 있어요...")

        all_similarity_results = [
                result
                for category in categories
                for result in calculate_pairwise_cosine_similarity(df_responses, loaded_aihub_embeddings_list, category)
            ]

        final_similarity_df_test = pd.DataFrame(all_similarity_results)

        similar_images = {
                category: final_similarity_df_test[final_similarity_df_test['response_column'] == category]
                .nlargest(3, 'cosine_similarity')
                .reset_index(drop=True)
                for category in categories
            }

        # 세션 상태 초기화
        for key, default in {
            "selected_images": {category: None for category in categories},
            "form_submitted": False,
            "profile_submitted": False,
            "new_traveler": None,
            "num_recommendations": 5,
            "user_profile": {}
        }.items():
            if key not in st.session_state:
                st.session_state[key] = default

        message_placeholder.write("비슷한 사진을 찾았어요! 마음에 드는 사진을 골라주세요.")

        # 이미지 선택 폼
        if not st.session_state['form_submitted']:
            with st.form("selection_form"):
                for category in categories:
                    display_name = category_names.get(category, category)
                    st.write(f"{display_name}에 대한 유사한 이미지입니다.")
                    cols = st.columns(3)
                    options = []
                    for i, (col, (_, row)) in enumerate(zip(cols, similar_images[category].iterrows())):
                        filename = row['aihub_image']
                        image_path = os.path.join(image_directory, filename)
                        with col:
                            if os.path.exists(image_path):
                                img = Image.open(image_path).resize((200, 200))
                                st.image(img, caption=f"{i + 1}", use_container_width=True)
                            else:
                                st.write(f"{i + 1}. 이미지 파일을 찾을 수 없습니다: {filename}")
                            options.append((f"{i + 1}", filename))
                    # 라디오 버튼을 통해 이미지 선택, 각 라디오의 키는 카테고리와 인덱스를 조합해 유일하게 설정
                    selected_option = st.radio(
                        "마음에 드는 사진을 골라주세요.",
                        [opt[0] for opt in options],
                        key = f"{category}_radio_{i}",
                        horizontal = True
                    )
                    # 선택된 파일명을 session_state에 저장
                    try:
                        st.session_state.selected_images[category] = next(
                            filename for label, filename in options if label == selected_option
                        )
                    except StopIteration:
                        st.write("선택된 이미지가 없습니다.")
                if st.form_submit_button("선택 완료"):
                    st.session_state.form_submitted = True
        message_placeholder.write("모든 선택을 완료했어요! 이제 최종 사진을 보여드릴게요.")
        # 폼 제출 후 최종 유사 이미지 계산
        if st.session_state["form_submitted"] and not st.session_state["profile_submitted"]:
            # 유사도 정보 생성 및 평균 유사도 계산
            average_similarities = generate_similarity_info(
                    st.session_state.selected_images,
                    final_similarity_df_test,
                    similar_images
                )
            # 가장 유사한 이미지 경로 가져오기
            most_similar_image_path = get_most_similar_image_path(average_similarities)
            if most_similar_image_path and os.path.exists(most_similar_image_path):
                message_placeholder.write("가장 비슷한 추억을 찾았어요!")
                most_similar_image = os.path.basename(most_similar_image_path)
                image = Image.open(most_similar_image_path).resize((250, 300))
                st.image(image)
                # 조건에 맞는 recommend_table 행 추출
                spot = recommend_table.loc[recommend_table['PHOTO_FILE_NM'] == most_similar_image]
                st.write("최종 사진 여행지 정보:", spot)

                # 사용자 특성 입력 폼
                travel_style_mapping = {
                    '자연 매우 선호': 1,
                    '자연 중간 선호': 2,
                    '자연 약간 선호': 3,
                    '중립': 4,
                    '도시 약간 선호': 5,
                    '도시 중간 선호': 6,
                    '도시 매우 선호': 7
                }

                travel_motive_mapping = {
                    '일상적인 환경 및 역할에서의 탈출, 지루함 탈피': 1,
                    '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식': 2,
                    '여행 동반자와의 친밀감 및 유대감 증진': 3,
                    '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기': 4,
                    'SNS 사진 등록 등 과시': 5,
                    '운동, 건강 증진 및 충전': 6,
                    '새로운 경험 추구': 7,
                    '역사 탐방, 문화적 경험 등 교육적 동기': 8,
                    '특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)': 9,
                    '기타': 10
                }

                st.subheader("본인 정보를 입력해 주세요")
                with st.form("user_profile_form"):
                    gender = st.radio("성별", [0, 1], format_func=lambda x: "남성" if x == 0 else "여성",
                                          horizontal=True)
                    age_group_label = st.radio("나이",
                     ['20대', '30대', '40대', '50대', '60대'], horizontal=True)
                    age_group = int(age_group_label[:-1])
                    travel_style_label = st.radio("자연 vs 도시", list(travel_style_mapping.keys()), horizontal=True)
                    travel_style = travel_style_mapping[travel_style_label]
                    travel_motive_label = st.radio("여행 동기", list(travel_motive_mapping.keys()), horizontal=True)
                    travel_motive = travel_motive_mapping[travel_motive_label]
                    num_recommendations = st.slider("추천받고 싶은 장소 개수", min_value=3, max_value=10, value=5, step=1)

                    if st.form_submit_button("선택 완료"):
                        st.session_state.user_profile = {
                                "GENDER": gender,
                                "AGE_GRP": age_group,
                                "TRAVEL_STYL_1": travel_style,
                                "TRAVEL_MOTIVE_1": travel_motive,
                                "NUM_RECOMMENDATIONS": num_recommendations
                            }
                        st.session_state.profile_submitted = True
                        st.write("사용자 프로필이 저장되었습니다.")
                        st.session_state.new_traveler = np.array([gender, age_group, travel_style, travel_motive])
                        st.session_state.num_recommendations = num_recommendations

        # 추천 장소 및 추가 정보 추출
        if st.session_state.profile_submitted:
            locations = recommend(
                        spot,
                        st.session_state.new_traveler,
                        metric = 'knn',
                        n = st.session_state.num_recommendations,
                        recommend_with_consume = recommend_with_consume
                    )
            locations_time = locations_with_time(locations, act_consume)
            dining = extract_dining(locations_time, act_consume)
            st.write("추천 장소 정보:", locations_time)

            # 경로 최적화 수행
            path, duration, dist, duration_matrix, _ = path_optimizer(spot, locations)

            # dining을 MultiIndex DataFrame으로 변환
            dining_multiindex_df = convert_dining_to_multiindex(dining)
            poi_store_mapping = dining_multiindex_df.groupby('POI_NM')['STORE_NM'].apply(list).reset_index()
            st.dataframe(poi_store_mapping)

            # 경로와 마커가 포함된 지도 생성
            route_map = create_map_with_markers_and_ordered_paths(path, poi_store_mapping)
            # Streamlit에 Folium 지도 표시
            st.write("최적 경로 지도")
            st_folium(route_map, width=700, height=500)

if __name__=='__main__' :
    main()

Overwriting app.py


## 초기로 돌아가자

In [69]:
%%writefile app.py

import os
import streamlit as st
from datetime import datetime
from PIL import Image, ImageOps
import pandas as pd
import numpy as np
# config.py에서 가져오기
from config import (
    load_model,
    calculate_pairwise_cosine_similarity,
    generate_similarity_info,
    get_most_similar_image_path,
    loaded_aihub_embeddings_list,
    image_directory,
    recommend_table,
    locations_with_time,
    extract_dining,
    recommend,
    path_optimizer,
    spot_table,
    act_consume,
    recommend_with_consume,
    convert_dining_to_multiindex,
    create_map_with_markers_and_ordered_paths
)
from streamlit_folium import st_folium

# 모델 로드 (캐시 사용)
processor, model1, model2 = load_model()


# 유저 업로드 이미지 저장 함수
def save_uploaded_file(directory, file) :
    if not os.path.exists(directory) :
        os.makedirs(directory)
    with open(os.path.join(directory, file.name), 'wb') as f:
        f.write(file.getbuffer())
    return os.path.join(directory, file.name)


# 프롬프트에 대한 응답 생성 함수
def generate_responses(image_path):
    prompts = [
        "USER: <image>\n What other travel destinations or travel activities do you have that go well with this place? Please list it in words. \nASSISTANT:",
        "USER: <image>\n Please express the season that goes best with this picture and why. \nASSISTANT:",
        "USER: <image>\n Tell me about the mood, touch and feelings of this image in detail. Use as many adjectives as possible to express them. Reduce objective descriptions and add a lot of subjective expressions. \nASSISTANT:",
        "USER: <image>\n Describe this image objectively and realistically using the objects in this image. Do not include subjective words such as the mood, touch, atmosphere and feelings. \nASSISTANT:"
        ]
    responses = []
    image = Image.open(image_path)

    for prompt in prompts:
        inputs = processor([image], [prompt], padding=True, return_tensors="pt").to("cuda")
        output = model1.generate(**inputs, max_new_tokens=100)
        generated_text = processor.batch_decode(output, skip_special_tokens=True)
        response = generated_text[0].split("ASSISTANT:")[-1].strip() if "ASSISTANT:" in generated_text[0] else generated_text[0].strip()
        responses.append(response)

    return responses

# 메인 함수
def main():
    st.title('사진 속 추억과 다시 만나는 여행✨')  # 웹사이트 메인 제목
    file = st.file_uploader('여행 사진을 업로드해 주세요.', type=['jpg','jpeg','png'])

    if file is not None :
        current_time = datetime.now().isoformat().replace(':', '_')
        file.name = current_time + '.jpg'
        image_path = save_uploaded_file('tmp', file)  # 코랩 좌측 열면 tmp라는 폴더가 생기고 거기에 사진 저장됨

        # 임시 공간 생성 및 메시지 표시
        message_placeholder = st.empty()

        image = Image.open(file)
        image = ImageOps.exif_transpose(image)  # EXIF 정보를 이용해 올바른 방향으로 회전
        st.image(image.resize((200, 220)))


        message_placeholder.write("🕵️사진을 분석하는 중이에요!")

        responses = generate_responses(image_path)
        categories = ['response_1', 'response_2', 'response_3', 'response_4']
        category_names = {
                'response_1': '비슷한 여행지',
                'response_2': '계절',
                'response_3': '감정',
                'response_4': '객관적 묘사'
            }
        df_responses = pd.DataFrame([responses], columns=categories)
        df_responses['image'] = file.name

        message_placeholder.write("🔎비슷한 사진을 찾고 있어요...")

        all_similarity_results = [
                result
                for category in categories
                for result in calculate_pairwise_cosine_similarity(df_responses, loaded_aihub_embeddings_list, category)
            ]

        final_similarity_df_test = pd.DataFrame(all_similarity_results)

        similar_images = {
                category: final_similarity_df_test[final_similarity_df_test['response_column'] == category]
                .nlargest(3, 'cosine_similarity')
                .reset_index(drop=True)
                for category in categories
            }

        # 세션 상태 초기화
        for key, default in {
            "selected_images": {category: None for category in categories},
            "form_submitted": False,
            "profile_submitted": False,
            "new_traveler": None,
            "num_recommendations": 5,
            "user_profile": {}
        }.items():
            if key not in st.session_state:
                st.session_state[key] = default

        message_placeholder.write("💖마음에 드는 사진을 골라주세요.")

        # 이미지 선택 폼
        if not st.session_state['form_submitted']:
            with st.form("selection_form"):
                for category in categories:
                    display_name = category_names.get(category, category)
                    st.write(f"✅{display_name}")
                    cols = st.columns(3)
                    options = []
                    for i, (col, (_, row)) in enumerate(zip(cols, similar_images[category].iterrows())):
                        filename = row['aihub_image']
                        image_path = os.path.join(image_directory, filename)
                        with col:
                            if os.path.exists(image_path):
                                img = Image.open(image_path)
                                img = ImageOps.exif_transpose(img)
                                img = img.resize((200, 200))
                                st.image(img, caption=f"{i + 1}", use_container_width=True)
                            else:
                                st.write(f"{i + 1}. 이미지 파일을 찾을 수 없습니다: {filename}")
                            options.append((f"{i + 1}", filename))
                    # 라디오 버튼을 통해 이미지 선택, 각 라디오의 키는 카테고리와 인덱스를 조합해 유일하게 설정
                    selected_option = st.radio(
                        "💖마음에 드는 사진을 골라주세요.",
                        [opt[0] for opt in options],
                        key = f"{category}_radio_{i}",
                        horizontal = True
                    )
                    # 선택된 파일명을 session_state에 저장
                    try:
                        st.session_state.selected_images[category] = next(
                            filename for label, filename in options if label == selected_option
                        )
                    except StopIteration:
                        st.write("선택된 이미지가 없습니다.")
                if st.form_submit_button("선택 완료"):
                    st.session_state.form_submitted = True
        message_placeholder.write("모든 선택을 완료했어요! 이제 최종 사진을 보여드릴게요.")
        # 폼 제출 후 최종 유사 이미지 계산
        if st.session_state["form_submitted"] and not st.session_state["profile_submitted"]:
            # 유사도 정보 생성 및 평균 유사도 계산
            average_similarities = generate_similarity_info(
                    st.session_state.selected_images,
                    final_similarity_df_test,
                    similar_images
                )
            # 가장 유사한 이미지 경로 가져오기
            most_similar_image_path = get_most_similar_image_path(average_similarities)
            if most_similar_image_path and os.path.exists(most_similar_image_path):
                message_placeholder.write("🎉가장 비슷한 추억을 찾았어요!")
                most_similar_image = os.path.basename(most_similar_image_path)
                img = Image.open(most_similar_image_path)
                img = ImageOps.exif_transpose(img)
                img = img.resize((200, 220))
                st.image(img)
                # 조건에 맞는 recommend_table 행 추출
                spot = recommend_table.loc[recommend_table['PHOTO_FILE_NM'] == most_similar_image]
                #st.write("최종 사진 여행지 정보:", spot)

                # 사용자 특성 입력 폼
                travel_style_mapping = {
                    '자연 매우 선호': 1,
                    '자연 중간 선호': 2,
                    '자연 약간 선호': 3,
                    '중립': 4,
                    '도시 약간 선호': 5,
                    '도시 중간 선호': 6,
                    '도시 매우 선호': 7
                }

                travel_motive_mapping = {
                    '일상적인 환경 및 역할에서의 탈출, 지루함 탈피': 1,
                    '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식': 2,
                    '여행 동반자와의 친밀감 및 유대감 증진': 3,
                    '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기': 4,
                    'SNS 사진 등록 등 과시': 5,
                    '운동, 건강 증진 및 충전': 6,
                    '새로운 경험 추구': 7,
                    '역사 탐방, 문화적 경험 등 교육적 동기': 8,
                    '특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)': 9,
                    '기타': 10
                }

                st.subheader("본인 정보를 입력해 주세요")
                with st.form("user_profile_form"):
                    gender = st.radio("✅성별", [0, 1], format_func=lambda x: "남성" if x == 0 else "여성",
                                          horizontal=True)
                    age_group_label = st.radio("✅나이",
                     ['20대', '30대', '40대', '50대', '60대'], horizontal=True)
                    age_group = int(age_group_label[:-1])
                    travel_style_label = st.radio("✅자연 vs 도시", list(travel_style_mapping.keys()))
                    travel_style = travel_style_mapping[travel_style_label]
                    travel_motive_label = st.radio("✅여행 동기", list(travel_motive_mapping.keys()))
                    travel_motive = travel_motive_mapping[travel_motive_label]
                    num_recommendations = st.slider("✅추천받고 싶은 장소 개수", min_value=3, max_value=10, value=5, step=1)

                    if st.form_submit_button("선택 완료"):
                        st.session_state.user_profile = {
                                "GENDER": gender,
                                "AGE_GRP": age_group,
                                "TRAVEL_STYL_1": travel_style,
                                "TRAVEL_MOTIVE_1": travel_motive,
                                "NUM_RECOMMENDATIONS": num_recommendations
                            }
                        st.session_state.profile_submitted = True
                        st.write("💌정보가 저장되었어요!")
                        st.session_state.new_traveler = np.array([gender, age_group, travel_style, travel_motive])
                        st.session_state.num_recommendations = num_recommendations

        # 추천 장소 및 추가 정보 추출
        if st.session_state.profile_submitted:
            locations = recommend(
                        spot,
                        st.session_state.new_traveler,
                        metric = 'knn',
                        n = st.session_state.num_recommendations,
                        recommend_with_consume = recommend_with_consume
                    )
            locations_time = locations_with_time(locations, act_consume)
            locations_time_print = locations_time[['VISIT_AREA_NM',
                                                   'ROAD_NM_ADDR',
                                                   'ratings']]
            locations_time_print.columns = ['이름', '주소', '별점']
            dining = extract_dining(locations_time, act_consume)
            st.subheader("✨추천 장소 정보")
            st.table(locations_time_print)

            # 경로 최적화 수행
            path, duration, dist, duration_matrix, _ = path_optimizer(spot, locations)

            # dining을 MultiIndex DataFrame으로 변환
            dining_multiindex_df = convert_dining_to_multiindex(dining)
            poi_store_mapping = dining_multiindex_df.groupby('POI_NM')['STORE_NM'].apply(list).reset_index()
            st.subheader("✨맛집 정보")
            poi_store_mapping_print = poi_store_mapping.copy()
            poi_store_mapping_print.columns = ['추천 장소', '주변 맛집']
            st.table(poi_store_mapping_print)

            # 경로와 마커가 포함된 지도 생성
            route_map = create_map_with_markers_and_ordered_paths(path, poi_store_mapping)
            # Streamlit에 Folium 지도 표시
            #st.write("최적 경로 지도")
            #st_folium(route_map, width=700, height=500)

if __name__=='__main__' :
    main()

Overwriting app.py


# streamlit 실행

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# "Password/Enpoint IP for localtunnel is:" 우측에 xx.xxx.xx.xxx 혹은 xx.xxx.xxx.xxx 형식의 숫자가 나온다.

!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 34.126.107.92
your url is: https://lazy-dogs-suffer.loca.lt


In [ ]:
!npm audit fix --force

npm WARN using --force Recommended protections disabled.
npm WARN audit Updating localtunnel to 1.8.3, which is a SemVer major change.
npm WARN deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
npm WARN deprecated sntp@1.0.9: This module moved to @hapi/sntp. Please make sure to switch over as this distribution is no longer supported and may contain bugs and critical security issues.
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated boom@2.10.1: This version has been deprecated in accordance with the hapi support policy (hapi.

# streamlit 코랩 연동 방법 (노건들)

In [ ]:
# 1. 스트림릿 설치
!pip install -q streamlit

In [ ]:
# 2_스트림릿 화면에 띄울 내용

%%writefile app.py

import streamlit as st

st.write('Hello, *World!* :sunglasses:') # 해당 내용을 수정해서 사이트를 자유롭게 꾸밀 수 있다.

In [ ]:
# 3
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# "Password/Enpoint IP for localtunnel is:" 우측에 xx.xxx.xx.xxx 혹은 xx.xxx.xxx.xxx 형식의 숫자가 나온다.

In [ ]:
# 4
!npm install localtunnel

In [ ]:
# 5
!streamlit run app.py &>/content/logs.txt &

In [ ]:
# 6
!npx localtunnel --port 8501

# "your url is:" 우측에 사이트 주소가 생성된다.